In [1]:
import pandas as pd  # to store data as dataframe
import numpy as np  # for numerical calculations such as histogramming
import matplotlib.pyplot as plt  # for plotting

In [2]:
np.__version__

'1.26.2'

In [3]:
seed_value = 420  # 42 is the answer to life, the universe and everything
from numpy.random import seed  # import the function to set the random seed in NumPy

seed(seed_value)  # set the seed value for random numbers in NumPy

In [4]:
# In this notebook we only process the main signal ggH125_ZZ4lep and the main background llll,
# for illustration purposes.
# You can add other backgrounds after if you wish.
samples = ["llll", "ggH125_ZZ4lep"]

In [5]:
# get data from files

DataFrames = {}  # define empty dictionary to hold dataframes
for s in samples:  # loop over samples
    DataFrames[s] = pd.read_csv("4lepton/" + s + ".csv")  # read .csv file

DataFrames["ggH125_ZZ4lep"]  # print signal data to take a look

,entry,lep_pt_0,lep_pt_1,lep_pt_2,lep_pt_3,lep_eta_0,lep_eta_1,lep_eta_2,lep_eta_3,lep_phi_0,...,lep_ptconerel_3,lep_etconerel_0,lep_etconerel_1,lep_etconerel_2,lep_etconerel_3,min_mll,mZ1,mZ2,mllll,totalWeight
0,0,51.905457,41.248570,16.397670,7.471228,-0.925709,-0.823695,-0.486417,0.266718,-2.774290,...,0.000000,-0.005114,0.034386,0.071266,-0.115843,9.953831,92.628607,23.311568,122.689157,0.000018
1,1,41.430645,40.307168,16.133789,7.481857,-1.233182,-0.396434,-0.541508,-0.302179,0.449031,...,0.151779,-0.030370,0.017931,0.030408,-0.120944,17.821408,86.697751,17.821408,113.659172,0.000034
2,2,33.646711,27.313271,20.035949,16.472641,-0.032324,-0.044153,0.067013,1.859531,-2.077223,...,0.000000,0.016144,-0.012675,0.101923,0.017772,32.122807,62.627431,32.122807,125.399958,0.000040
3,3,77.118562,27.845740,17.726541,14.714521,0.514764,0.845311,2.189158,0.179711,0.115437,...,0.000000,-0.005004,-0.007781,0.000000,0.034991,29.143731,84.535069,29.143731,123.645105,0.000059
4,4,161.909219,53.367754,25.596689,18.864479,-1.037354,-0.821728,-1.261883,0.126195,-0.161717,...,0.000000,0.018625,-0.000264,-0.001246,-0.002172,39.159591,68.958081,48.500839,123.518264,0.000078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164711,164711,32.143482,24.158068,17.203547,14.358152,-1.003832,0.609448,0.876340,1.039745,-0.417631,...,0.000000,-0.020921,0.040304,-0.001333,0.010283,25.926057,62.697352,39.612517,123.352912,0.000019
164712,164712,39.488273,33.694094,32.709998,14.797520,0.184790,0.799441,-0.454989,-1.167309,-0.747386,...,0.000000,0.040552,-0.000592,0.018882,-0.019414,30.654450,56.876117,30.654450,122.652688,0.000054
164713,164713,63.284211,22.707840,15.635994,14.873250,0.935591,0.184483,0.174508,2.128866,-1.503719,...,0.000000,0.010497,-0.015776,-0.022911,-0.003570,30.952590,66.649322,37.686620,124.312299,0.000071
164714,164714,52.538805,40.321457,25.766850,19.381920,0.880250,1.205615,1.701138,0.853039,-2.060255,...,0.000000,0.017046,-0.013661,0.057185,0.000224,19.868087,88.412652,19.868087,124.105421,0.000060


In [6]:
# cut on lepton type
def cut_lep_type(lep_type_0, lep_type_1, lep_type_2, lep_type_3):
    # first lepton is [0], 2nd lepton is [1] etc
    # for an electron lep_type is 11
    # for a muon lep_type is 13
    # only want to keep events where one of eeee, mumumumu, eemumu
    sum_lep_type = lep_type_0 + lep_type_1 + lep_type_2 + lep_type_3
    if sum_lep_type == 44 or sum_lep_type == 48 or sum_lep_type == 52:
        return True
    else:
        return False

In [7]:
# apply cut on lepton type
for s in samples:
    # cut on lepton type using the function cut_lep_type defined above
    DataFrames[s] = DataFrames[s][
        np.vectorize(cut_lep_type)(
            DataFrames[s].lep_type_0,
            DataFrames[s].lep_type_1,
            DataFrames[s].lep_type_2,
            DataFrames[s].lep_type_3,
        )
    ]
DataFrames["ggH125_ZZ4lep"]  # print signal data to take a look

,entry,lep_pt_0,lep_pt_1,lep_pt_2,lep_pt_3,lep_eta_0,lep_eta_1,lep_eta_2,lep_eta_3,lep_phi_0,...,lep_ptconerel_3,lep_etconerel_0,lep_etconerel_1,lep_etconerel_2,lep_etconerel_3,min_mll,mZ1,mZ2,mllll,totalWeight
0,0,51.905457,41.248570,16.397670,7.471228,-0.925709,-0.823695,-0.486417,0.266718,-2.774290,...,0.000000,-0.005114,0.034386,0.071266,-0.115843,9.953831,92.628607,23.311568,122.689157,0.000018
1,1,41.430645,40.307168,16.133789,7.481857,-1.233182,-0.396434,-0.541508,-0.302179,0.449031,...,0.151779,-0.030370,0.017931,0.030408,-0.120944,17.821408,86.697751,17.821408,113.659172,0.000034
2,2,33.646711,27.313271,20.035949,16.472641,-0.032324,-0.044153,0.067013,1.859531,-2.077223,...,0.000000,0.016144,-0.012675,0.101923,0.017772,32.122807,62.627431,32.122807,125.399958,0.000040
3,3,77.118562,27.845740,17.726541,14.714521,0.514764,0.845311,2.189158,0.179711,0.115437,...,0.000000,-0.005004,-0.007781,0.000000,0.034991,29.143731,84.535069,29.143731,123.645105,0.000059
4,4,161.909219,53.367754,25.596689,18.864479,-1.037354,-0.821728,-1.261883,0.126195,-0.161717,...,0.000000,0.018625,-0.000264,-0.001246,-0.002172,39.159591,68.958081,48.500839,123.518264,0.000078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164711,164711,32.143482,24.158068,17.203547,14.358152,-1.003832,0.609448,0.876340,1.039745,-0.417631,...,0.000000,-0.020921,0.040304,-0.001333,0.010283,25.926057,62.697352,39.612517,123.352912,0.000019
164712,164712,39.488273,33.694094,32.709998,14.797520,0.184790,0.799441,-0.454989,-1.167309,-0.747386,...,0.000000,0.040552,-0.000592,0.018882,-0.019414,30.654450,56.876117,30.654450,122.652688,0.000054
164713,164713,63.284211,22.707840,15.635994,14.873250,0.935591,0.184483,0.174508,2.128866,-1.503719,...,0.000000,0.010497,-0.015776,-0.022911,-0.003570,30.952590,66.649322,37.686620,124.312299,0.000071
164714,164714,52.538805,40.321457,25.766850,19.381920,0.880250,1.205615,1.701138,0.853039,-2.060255,...,0.000000,0.017046,-0.013661,0.057185,0.000224,19.868087,88.412652,19.868087,124.105421,0.000060


In [8]:
# cut on lepton charge
def cut_lep_charge(lep_charge_0,lep_charge_1,lep_charge_2,lep_charge_3):
# only want to keep events where sum of lepton charges is 0
    sum_lep_charge = lep_charge_0 + lep_charge_1 + lep_charge_2 + lep_charge_3
    if sum_lep_charge==0: return True
    else: return False

# apply cut on lepton charge
for s in samples:
    # cut on lepton charge using the function cut_lep_charge defined above
    DataFrames[s] = DataFrames[s][ np.vectorize(cut_lep_charge)(DataFrames[s].lep_charge_0,
                                                    	    DataFrames[s].lep_charge_1,
                                                    	    DataFrames[s].lep_charge_2,
                                                    	    DataFrames[s].lep_charge_3) ]
DataFrames['ggH125_ZZ4lep'] # print signal data to take a look

,entry,lep_pt_0,lep_pt_1,lep_pt_2,lep_pt_3,lep_eta_0,lep_eta_1,lep_eta_2,lep_eta_3,lep_phi_0,...,lep_ptconerel_3,lep_etconerel_0,lep_etconerel_1,lep_etconerel_2,lep_etconerel_3,min_mll,mZ1,mZ2,mllll,totalWeight
0,0,51.905457,41.248570,16.397670,7.471228,-0.925709,-0.823695,-0.486417,0.266718,-2.774290,...,0.000000,-0.005114,0.034386,0.071266,-0.115843,9.953831,92.628607,23.311568,122.689157,0.000018
1,1,41.430645,40.307168,16.133789,7.481857,-1.233182,-0.396434,-0.541508,-0.302179,0.449031,...,0.151779,-0.030370,0.017931,0.030408,-0.120944,17.821408,86.697751,17.821408,113.659172,0.000034
2,2,33.646711,27.313271,20.035949,16.472641,-0.032324,-0.044153,0.067013,1.859531,-2.077223,...,0.000000,0.016144,-0.012675,0.101923,0.017772,32.122807,62.627431,32.122807,125.399958,0.000040
3,3,77.118562,27.845740,17.726541,14.714521,0.514764,0.845311,2.189158,0.179711,0.115437,...,0.000000,-0.005004,-0.007781,0.000000,0.034991,29.143731,84.535069,29.143731,123.645105,0.000059
4,4,161.909219,53.367754,25.596689,18.864479,-1.037354,-0.821728,-1.261883,0.126195,-0.161717,...,0.000000,0.018625,-0.000264,-0.001246,-0.002172,39.159591,68.958081,48.500839,123.518264,0.000078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164711,164711,32.143482,24.158068,17.203547,14.358152,-1.003832,0.609448,0.876340,1.039745,-0.417631,...,0.000000,-0.020921,0.040304,-0.001333,0.010283,25.926057,62.697352,39.612517,123.352912,0.000019
164712,164712,39.488273,33.694094,32.709998,14.797520,0.184790,0.799441,-0.454989,-1.167309,-0.747386,...,0.000000,0.040552,-0.000592,0.018882,-0.019414,30.654450,56.876117,30.654450,122.652688,0.000054
164713,164713,63.284211,22.707840,15.635994,14.873250,0.935591,0.184483,0.174508,2.128866,-1.503719,...,0.000000,0.010497,-0.015776,-0.022911,-0.003570,30.952590,66.649322,37.686620,124.312299,0.000071
164714,164714,52.538805,40.321457,25.766850,19.381920,0.880250,1.205615,1.701138,0.853039,-2.060255,...,0.000000,0.017046,-0.013661,0.057185,0.000224,19.868087,88.412652,19.868087,124.105421,0.000060


In [9]:
lep_pt_2 = {  # dictionary containing plotting parameters for the lep_pt_2 histogram
    # change plotting parameters
    "bin_width": 1,  # width of each histogram bin
    "num_bins": 13,  # number of histogram bins
    "xrange_min": 7,  # minimum on x-axis
    "xlabel": r"$lep\_pt$[2] [GeV]",  # x-axis label
}

In [10]:
lep_pt_1 = { # dictionary containing plotting parameters for the lep_pt_1 histogram
    # change plotting parameters
    'bin_width':1, # width of each histogram bin
    'num_bins':28, # number of histogram bins
    'xrange_min':7, # minimum on x-axis
    'xlabel':r'$lep\_pt$[1] [GeV]', # x-axis label
}

In [11]:
SoverB_hist_dict = {
    "lep_pt_2": lep_pt_2,
    "lep_pt_1": lep_pt_1,
}  # add a histogram here if you want it plotted

In [ ]:
from my_functions import plot_SoverB

plot_SoverB(DataFrames, SoverB_hist_dict)

: 